<a href="https://colab.research.google.com/github/Tarunparkar/ML/blob/main/ML3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Step 2: Load Boston Housing dataset
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)

X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]

# Step 3: Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Add bias term (intercept)
X_train_scaled = np.c_[np.ones(X_train_scaled.shape[0]), X_train_scaled]
X_test_scaled = np.c_[np.ones(X_test_scaled.shape[0]), X_test_scaled]

print("Data prepared.")

# Step 5: Evaluation function
def evaluate_model(X_train, y_train, X_test, y_test, theta, model_name="Model"):
    y_train_pred = X_train @ theta
    y_test_pred = X_test @ theta

    mse_train = mean_squared_error(y_train, y_train_pred)
    r2_train = r2_score(y_train, y_train_pred)
    error_train = np.mean(np.abs((y_train - y_train_pred) / y_train)) * 100

    mse_test = mean_squared_error(y_test, y_test_pred)
    r2_test = r2_score(y_test, y_test_pred)
    error_test = np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100

    print(f"\n{model_name}:")
    print(f" Train -> MSE: {mse_train:.4f}, R²: {r2_train:.4f}, Error %: {error_train:.2f}%")
    print(f" Test  -> MSE: {mse_test:.4f}, R²: {r2_test:.4f}, Error %: {error_test:.2f}%")

    return {
        "Model": model_name,
        "MSE Train": mse_train,
        "R2 Train": r2_train,
        "Error % Train": error_train,
        "MSE Test": mse_test,
        "R2 Test": r2_test,
        "Error % Test": error_test,
    }

# Step 6: Gradient Descent implementations

def gradient_descent_vanilla(X, y, lr=0.01, n_iters=5000):
    m, n = X.shape
    theta = np.zeros(n)
    for _ in range(n_iters):
        y_pred = X @ theta
        error = y_pred - y
        gradients = (1/m) * X.T @ error
        theta -= lr * gradients
    return theta

def gradient_descent_l2(X, y, lr=0.01, n_iters=5000, lambda_=0.1):
    m, n = X.shape
    theta = np.zeros(n)
    for _ in range(n_iters):
        y_pred = X @ theta
        error = y_pred - y
        gradients = (1/m) * (X.T @ error + lambda_ * np.r_[0, theta[1:]])  # Don't regularize bias
        theta -= lr * gradients
    return theta

def gradient_descent_l1(X, y, lr=0.01, n_iters=5000, lambda_=0.1):
    m, n = X.shape
    theta = np.zeros(n)
    for _ in range(n_iters):
        y_pred = X @ theta
        error = y_pred - y
        gradients = (1/m) * X.T @ error
        gradients[1:] += lambda_ * np.sign(theta[1:])
        theta -= lr * gradients
    return theta

def gradient_descent_elasticnet(X, y, lr=0.01, n_iters=5000, lambda1=0.1, lambda2=0.1):
    m, n = X.shape
    theta = np.zeros(n)
    for _ in range(n_iters):
        y_pred = X @ theta
        error = y_pred - y
        gradients = (1/m) * X.T @ error
        gradients[1:] += lambda1 * np.sign(theta[1:]) + lambda2 * theta[1:]
        theta -= lr * gradients
    return theta

# Step 7: Train and evaluate all models

results = []

# Vanilla Linear Regression
theta_vanilla = gradient_descent_vanilla(X_train_scaled, y_train)
results.append(evaluate_model(X_train_scaled, y_train, X_test_scaled, y_test, theta_vanilla, "Vanilla Linear Regression"))

# L2 (Ridge)
theta_l2 = gradient_descent_l2(X_train_scaled, y_train, lambda_=0.1)
results.append(evaluate_model(X_train_scaled, y_train, X_test_scaled, y_test, theta_l2, "L2 Regularization (Ridge)"))

# L1 (Lasso)
theta_l1 = gradient_descent_l1(X_train_scaled, y_train, lambda_=0.1)
results.append(evaluate_model(X_train_scaled, y_train, X_test_scaled, y_test, theta_l1, "L1 Regularization (Lasso)"))

# ElasticNet (L1 + L2)
theta_elastic = gradient_descent_elasticnet(X_train_scaled, y_train, lambda1=0.1, lambda2=0.1)
results.append(evaluate_model(X_train_scaled, y_train, X_test_scaled, y_test, theta_elastic, "ElasticNet (L1 + L2)"))

# Step 8: Summary table
results_df = pd.DataFrame(results).set_index("Model")
print("\nSummary of All Models:")
print(results_df)


Data prepared.

Vanilla Linear Regression:
 Train -> MSE: 21.6424, R²: 0.7509, Error %: 16.56%
 Test  -> MSE: 24.3385, R²: 0.6681, Error %: 16.89%

L2 Regularization (Ridge):
 Train -> MSE: 21.6426, R²: 0.7509, Error %: 16.55%
 Test  -> MSE: 24.3402, R²: 0.6681, Error %: 16.89%

L1 Regularization (Lasso):
 Train -> MSE: 22.1548, R²: 0.7450, Error %: 16.25%
 Test  -> MSE: 25.6748, R²: 0.6499, Error %: 17.30%

ElasticNet (L1 + L2):
 Train -> MSE: 23.0175, R²: 0.7350, Error %: 16.28%
 Test  -> MSE: 25.9956, R²: 0.6455, Error %: 17.11%

Summary of All Models:
                           MSE Train  R2 Train  Error % Train   MSE Test  \
Model                                                                      
Vanilla Linear Regression  21.642408  0.750874      16.557457  24.338532   
L2 Regularization (Ridge)  21.642574  0.750872      16.553969  24.340177   
L1 Regularization (Lasso)  22.154796  0.744976      16.245284  25.674811   
ElasticNet (L1 + L2)       23.017466  0.735046      16.280